<a href="https://colab.research.google.com/github/rapp2043/comedian-mortality/blob/main/comedian_mortality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Web scraper program for list of comedians on Wikipedia
import requests
from bs4 import BeautifulSoup
import csv

def get_comedians_list(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    comedians = []

    # Find all the <a> tags within the <li> tags
    for li_tag in soup.find_all('li'):
        a_tag = li_tag.find('a')
        if a_tag and 'href' in a_tag.attrs:
            link = a_tag.attrs['href']
            if link.startswith('/wiki/'):
                # Construct the full URL
                full_url = "https://en.wikipedia.org" + link
                comedians.append({'name': a_tag.text, 'url': full_url})

    return comedians

def save_to_csv(data, filename):
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['name', 'url']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for row in data:
            writer.writerow(row)

# URL of the Wikipedia page
url = 'https://en.wikipedia.org/wiki/List_of_comedians'

# Get the list of comedians and their URLs
comedians_list = get_comedians_list(url)

# Save the data to a CSV file
save_to_csv(comedians_list, 'comedians.csv')

print("CSV file has been created with the list of comedians.")

CSV file has been created with the list of comedians.
